In [81]:
import cx_Oracle as cx
import oracledb
import pickle
import pandas as pd
import numpy as np


In [82]:
oracledb.init_oracle_client(lib_dir="/Users/minjoongkim/instantclient_19_8")

In [83]:
with open('data/world_bank_all.pkl', 'rb') as f:
    wb_df = pickle.load(f)
with open('data/global.pkl', 'rb') as f:
    global_df = pickle.load(f)

In [84]:
global_df['Year'] = global_df['Year'].astype(np.float64)

In [85]:
g_rows = list(global_df.to_records(index=False))

In [86]:
wb_df['date'] = wb_df['date'].astype(np.float64)

In [ ]:
wb_df['co2e'].fillna('', inplace=True)
wb_df['pm25'].fillna('', inplace=True)
wb_df['gdp'].fillna('', inplace=True)
wb_df['population'].fillna('', inplace=True)

In [91]:
# Column 순서 수정, DB Column 순서에 맞춰서 재배치
new_order = ['date', 'countryiso3code' , 'country_code', 'country', 'co2e', 'gdp', 'population', 'pm25']
wb_df = wb_df[new_order]

In [93]:
wb_df.index = range(1, len(wb_df) + 1)

In [94]:
wb_rows = list(wb_df.to_records())

In [95]:
wb_rows2 = []
for row in wb_rows:
    wb_rows2.append(list(row))

In [96]:
converted_list = [[float(sublist[0])] + sublist[1:] for sublist in wb_rows2]


In [99]:
type(converted_list)

list

In [ ]:
conn = cx.connect(user="lab02", password="lab02", dsn="localhost:1521/xe")
cursor = conn.cursor()
cursor.executemany("insert into global_data values(:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11)", list(g_rows))
cursor.execute('commit')
cursor.executemany("insert into country_data values(:1, :2, :3, :4, :5, :6, :7, :8, :9)", list(converted_list))
cursor.execute('commit')
conn.close()

In [114]:
conn = cx.connect(user="lab02", password="lab02", dsn="localhost:1521/xe")
cursor = conn.cursor()
cursor.execute("delete from country_data where \"year\" = 2021")
cursor.execute("delete from country_data where \"year\" < 1978")
cursor.execute('commit')
conn.close()

In [115]:
conn = cx.connect(user="lab02", password="lab02", dsn="localhost:1521/xe")
cursor = conn.cursor()
cursor.execute("alter table country_data add constraint fk_year foreign key (\"year\") references global_data(\"year\")")
cursor.execute('commit')
conn.close()